# DLKit Overview

DLKit is a Python implementation of the OSIDs (www.osid.org).

Every service has a catalog object, which houses / contains additional objects. These catalogs help control authorization to the contained objects, and can be arranged hierarchically (with authorizations flowing down).

             MIT
            /   \
           /     \
       Physics   Math  

Someone with access to all of MIT could see both Math and Physics materials, but folks in Physics may not be able to see Math materials.

Let's start with a simple example and dig more in depth.

## Assessment Service

In the Assessment Service, the catalog is called a `bank`. `banks` contain other objects, called `item`, `assessment`, `assessmentOffered`, `assessmentTaken`, etc. We'll start with `item` and `assessment`.

To being with, you access each service through a `manager`. For example, an `AssessmentManager` gives you access to the various `bank` and `item` methods.

In order to get a `manager`, we go through the `runtime` -- in this tutorial, the `dlkit_edx` runtime. We will simulate a user and a test web request, to pass along. This username is automatically included with many types of data, such as when taking assessments.

In [3]:
from dlkit_edx import PROXY_SESSION, RUNTIME
from dlkit_edx.proxy_example import TestRequest

condition = PROXY_SESSION.get_proxy_condition()
dummy_request = TestRequest(username='tutorial_user@school.edu',
                            authenticated=True)
condition.set_http_request(dummy_request)
proxy = PROXY_SESSION.get_proxy(condition)
am = RUNTIME.get_service_manager('ASSESSMENT',
                                  proxy=proxy)

print am

**Banks**

Now that we have an `AssessmentManager`, we can see what `banks` exist in the system. Calling for "lists" of things returns a Python generator, and we can check the number of results with `.available()`.


In [4]:
print am.banks
print am.banks.available()

0


No banks!! Okay, let's create one. In `DLKit`, CRUD operations are done with `forms`. So let's get a `form` to create a new assessment `bank`, and assign it a `displayName` and `description` that we'll recognize later.

Notice that in the `get_bank_form_for_create()` method, we pass an empty list as an argument. This can be used to extend the base functionality of the `bank`, via record extensions. That is a more advanced feature we'll touch on later.

In [5]:
form = am.get_bank_form_for_create([])
form.display_name = "MyBank"
form.description = "For learning about DLKit"
bank = am.create_bank(form)
print bank
print bank.display_name.text
print bank.description.text
print str(bank.ident)

MyBank
For learning about DLKit
assessment.Bank%3A57fe54dccdfc5c3a0bc23b47%40ODL.MIT.EDU


You can see that our new `bank` has some attributes -- some that we assigned (`displayName` and `description`, and others that were created by `DLKit`, like `ident`). `displayName` and `description` are actually objects themselves, and can also contain language, format, and script data, which is why we call `display_name.text` and `description.text` above.

##Items
An `item` is what you might think of as a basic assessment question with the associated answers (right or wrong). There are many types of `item`s, including multiple choice, fill in the blank, short answer, etc.